# 3. Swiss & Foreign Unemployement Rates

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import folium
import pandas as pd
folium.__version__ == '0.5.0'

True

### Map

Import the TopoJson Map for Switzerland. I also have converted the TopoJson file into a GeoJson file using the online converter : http://jeffpaine.github.io/geojson-topojson/  

In [2]:
state_geo_path = r'topojson\ch-cantons.json'
state_topo_path = r'topojson\ch-cantons.topojson.json'
canton_json_data = json.load(open(state_geo_path))
canton_topojson = json.load(open(state_topo_path))

FileNotFoundError: [Errno 2] No such file or directory: 'topojson\\ch-cantons.json'

### Export Data 

From amstat website : https://www.amstat.ch/v2/index.jsp I downloaded a .txt file with unemployement ratio per Swiss Canton and residents' nationality.

In [ ]:
nationality_rate = pd.read_csv('nationality_rate.txt', sep = ',',header = 1,\
                               names = ['Canton','population','Measure','Rate',\
                                        'Ch1','Taux de chômage.1','Ch1.1'])
nationality_rate.drop(['Measure','Taux de chômage.1','Ch1.1'], axis = 1, inplace = True)



nationality_rate.head()

### Format Data

In [ ]:
canton_abrev = ['ZH','ZG','VD','VS','UR','TI','TG','SG','SO','SZ','SH','OW','NW','NE','LU','JU','GR','GL',\
                'GE','FR','BE','BS','BL','AI','AR','AG']

In [ ]:
suisse = nationality_rate.copy()
suisse = suisse.loc[suisse['population'] == 'Suisses',:]
suisse.sort_values('Canton', ascending = False, inplace = True)
suisse['Abrev'] = canton_abrev
suisse.set_index('Abrev', drop = False, inplace = True)
suisse.head()

In [ ]:
foreign = nationality_rate.copy()
foreign = foreign.loc[foreign['population'] == 'Etrangers']
foreign.sort_values('Canton', ascending = False, inplace = True)
foreign['Abrev'] = canton_abrev
foreign.set_index('Abrev', drop = False, inplace = True)

We create below the datafrmae containing usefull information for map plots. It is composed of 26 rows, for each Canton. The columns contain unemployment rates for Swiss residents (1st column), Foreign residents (2nd Column), normalised difference between ratio (3rd column).

In [ ]:
foreign_suisse = pd.DataFrame(index = canton_abrev)
foreign_suisse['Foreign'] = foreign.sort_values('Canton', ascending = False).loc[:,'Rate']
foreign_suisse['Suisse'] = suisse.sort_values('Canton', ascending = False).loc[:,'Rate']
foreign_suisse['Diff'] = foreign_suisse['Foreign'].div(foreign_suisse['Suisse'])
foreign_suisse['Abrev'] = canton_abrev
foreign_suisse.head()

### Display Data
Define the order of the canton as in the TopoJson file 

In [ ]:
canton_order_map = [canton_topojson['objects']['cantons']['geometries'][i]['id'] for i in range(0,26)]

In [ ]:
def weight_function(canton):
    
    chomage = foreign_suisse.loc[canton,'Foreign']
    max_weight = 10
    min_weight = 1
    
    max_chomage = foreign_suisse['Foreign'].max()
    min_chomage = foreign_suisse['Foreign'].min()
    
    weight= int(round((chomage-min_chomage)/(max_chomage-min_chomage)*(max_weight-min_weight) + min_weight, 0))
    
    return weight

In [ ]:
m_foreign_suisse_ratio = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)


m_foreign_suisse_ratio.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons',
                                  data=foreign_suisse.loc[canton_order_map,:],
                                  columns=['Abrev', 'Suisse'],
                                  key_on='feature.id',
                                  fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9, line_weight = 1,
                                  legend_name='Percentage of swiss unemployment(%)')

folium.TopoJson(canton_topojson, 'objects.cantons', style_function = lambda feature:{
    'weight' : weight_function(feature['id']),
    'color'  : 'grey',
    'dashArray' : '1, 0'
}).add_to(m_foreign_suisse_ratio)

m_foreign_suisse_ratio.save('Foreign_suisse_ratio.html')

Here we have generated a map with both Swiss and Foreign unemployement ratio per canton, using a 'color_fill' differenciation for swiss ratio and an 'edge_weight' differenciation for foreign ratio (see : [Foreign Suisse Unemployement Ratio](Foreign_suisse_ratio.html)). This display is not optimal since I wasn't able to add legend for Foreign ratio feature, at least, it gives a relative overview considering the edges are ticker for larger ratios. 
In a more classic way, we can create two maps to display both features separately. See below

In [ ]:
m_suisse_ratio = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)

m_suisse_ratio.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons', data=foreign_suisse.loc[canton_order_map,:],
             columns=['Abrev', 'Suisse'],
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9,
             legend_name='Percentage of swiss unemployment(%)')

m_suisse_ratio.save('Suisse_ratio.html')

Here we have plot the [Suisse Unemployement Ratio](Suisse_ratio.html)

In [ ]:
m_foreign_ratio = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)

m_foreign_ratio.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons', data=foreign_suisse.loc[canton_order_map,:],
             columns=['Abrev', 'Foreign'],
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9,
             legend_name='Percentage of foreigners unemployment in switzerland (%)')


m_foreign_ratio.save('Foreign_ratio.html')

Here we have plot the [Foreign Unemployement Ratio](Foreign_ratio.html)

In [ ]:
m_diff_ratio = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)

m_diff_ratio.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons', data=foreign_suisse.loc[canton_order_map,:],
             columns=['Abrev', 'Diff'],
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9,
             legend_name='Ratio between Swiss and Foreign Unemployment ratios')

m_diff_ratio.save('Diff_ratio.html')

Finally, a good way to display the difference in unemployment rates between the two categories in each canton is to plot the [Ratio between Swiss and Foreign Unemployement rates](Diff_ratio.html)  
  
    
We observe in a first time that unemployment ratios for swiss residents are higher in the west along french boundary with highest values obtained in Geneva and Neuchâtel . In the same way, unemployment ratios for foreign residents is high along the french boundary but with highest values obtained in Jura. From the last map, We must notice the foreign unmployment ratio is usually more than twice the suisse unemployment ratio, the ratio between those two is higer in the center and east Switzerland.

# 3.2 Refined Analysis : addition of differences between age groups
### Export Data

From amstat website : https://www.amstat.ch/v2/index.jsp we downloaded a .txt file with neumber of unemployment workers per Swiss Canton, residents' nationality and age.

In [ ]:
age_nationality_nb = pd.read_csv('age_nationality_numbers_2.txt', sep = ',', header = 0, thousands = '\'', 
                                 names = ['Canton','Population','age class','Drop','Drop1','Chomeurs inscrits'])
age_nationality_nb.drop(0,axis = 0, inplace = True)
age_nationality_nb.drop(['Drop','Drop1'], axis = 1, inplace = True)
age_nationality_nb.dropna(axis = 0, inplace = True)
age_nationality_nb.drop(age_nationality_nb.loc[age_nationality_nb['age class'] == 'Total'].index , axis = 0, inplace = True)
age_nationality_nb.sort_values(['Canton','Population','age class'], ascending = False, inplace = True)

age_nationality_nb.head(10)

### Format Data for Display

In [ ]:
header = [np.array(["Suisse","Suisse","Suisse","Foreign","Foreign","Foreign"]),\
          np.array(['15-24','25-49','50+','15-24','25-49','50+'])] 


age_nationality_to_plot = pd.DataFrame( index = canton_abrev)
age_nationality_to_plot.index.name = 'Canton'

suisse_age_1 = list(age_nationality_nb.loc[(age_nationality_nb['age class'] == '1') & (age_nationality_nb['Population'] == 'Suisses')]\
                    ['Chomeurs inscrits'].str.replace('\'','').astype(float))
suisse_age_2 = list(age_nationality_nb.loc[(age_nationality_nb['age class'] == '2') & (age_nationality_nb['Population'] == 'Suisses')]\
                    ['Chomeurs inscrits'].str.replace('\'','').astype(float))
suisse_age_3 = list(age_nationality_nb.loc[(age_nationality_nb['age class'] == '3') & (age_nationality_nb['Population'] == 'Suisses')]\
                    ['Chomeurs inscrits'].str.replace('\'','').astype(float))

foreign_age_1 = list(age_nationality_nb.loc[(age_nationality_nb['age class'] == '1') & (age_nationality_nb['Population'] == 'Etrangers')]\
                     ['Chomeurs inscrits'].str.replace('\'','').astype(float))
foreign_age_2 = list(age_nationality_nb.loc[(age_nationality_nb['age class'] == '2') & (age_nationality_nb['Population'] == 'Etrangers')]\
                     ['Chomeurs inscrits'].str.replace('\'','').astype(float))
foreign_age_3 = list(age_nationality_nb.loc[(age_nationality_nb['age class'] == '3') & (age_nationality_nb['Population'] == 'Etrangers')]\
                     ['Chomeurs inscrits'].str.replace('\'','').astype(float))


age_nationality_to_plot['Ch1'] = suisse_age_1
age_nationality_to_plot['Ch2'] = suisse_age_2
age_nationality_to_plot['Ch3'] = suisse_age_3

age_nationality_to_plot['Ch4'] = foreign_age_1
age_nationality_to_plot['Ch5'] = foreign_age_2
age_nationality_to_plot['Ch6'] = foreign_age_3


age_nationality_to_plot.columns = header
age_nationality_to_plot.columns.names = ['Origin', 'Age']

age_nationality_to_plot.head()



### Display

In [ ]:
age_nationality_to_plot.iloc[0:13].plot(kind='bar',figsize=[20,8],fontsize=17, grid = True, \
                                        title='Unemployment in all 26 Swiss Cantons, for nationality class and age class',\
                                        colormap = 'bwr', legend = 'resident', rot = 0)
age_nationality_to_plot.iloc[14::].plot(kind='bar',figsize=[20,8],fontsize=17, grid = True,\
                                        title='Unemployment in all 26 Swiss Cantons, for nationality class and age class',\
                                        colormap = 'bwr', rot = 0)
plt.show()

The sheer numbers presented above don't say a lot. Still, we are able to identify the five main cantons with high industrial activity.    
Since we don't have access to the exact number of active people in each cathegory, we will make to different assumptions to approximate the true rates :
- First one, lets consider the active population in all class age is invariant with respect to the origin of the workers (foreign or swiss) in each canton.
- Second one, lets consider the active population in origin cathegory is invariant with respect to the age of the workers in each canton.

#### First assumption


In [ ]:
nationality_rate.loc[:,'Ch1'] = nationality_rate['Ch1'].str.replace('\'','').astype(float)
nationality_rate['Active'] = round(nationality_rate['Ch1'].div(nationality_rate['Rate'])*100,0)

suisse_active = list(nationality_rate.loc[(nationality_rate['population'] == 'Suisses')].sort_values('Canton',ascending = False)['Active'])
foreign_active = list(nationality_rate.loc[(nationality_rate['population'] == 'Etrangers')].sort_values('Canton',ascending = False)['Active'])

In [ ]:
assumption_1 = age_nationality_to_plot.copy()
assumption_1.columns = ['ch1','ch2','ch3','ch4','ch5','ch6']

assumption_1.loc[:,['ch1','ch2','ch3']] = assumption_1[['ch1','ch2','ch3']].div(suisse_active, axis =0)*3
assumption_1.loc[:,['ch4','ch5','ch6']] = assumption_1[['ch4','ch5','ch6']].div(foreign_active, axis =0)*3

assumption_1_to_plot = assumption_1.copy()
assumption_1_to_plot.columns = header
assumption_1_to_plot.columns.names = ['Origin', 'Age']

assumption_1_to_plot.head()

In [ ]:
assumption_1_to_plot.iloc[0:13].plot(kind='bar',figsize=[20,8],fontsize=17, grid = True, \
                                        title='Unemployment in all 26 Swiss Cantons, for nationality class and age class',\
                                        colormap = 'bwr', rot = 0)
assumption_1_to_plot.iloc[14::].plot(kind='bar',figsize=[20,8],fontsize=17, grid = True,\
                                        title='Unemployment in all 26 Swiss Cantons, for nationality class and age class',\
                                        colormap = 'bwr', rot = 0)
plt.show()

From this plot, we can identify global unemployment tendency. We notice unemployment rates are higher for foreign residents, we believe the unemployment rates are higher for middle age population regardless of the origin of the resident. Of course this model is very limited because the range '25-49' might have the largest working population. 

#### Assumption 2

In [ ]:
age_ratio = pd.read_csv('age_ratio_2.txt', sep = ',', header = 1,names = ['age class','drop1','drop2','Rate','Ch1','drop3','drop4'])
age_ratio.drop(['drop1','drop2','drop3','drop4'], axis = 1, inplace = True)
age_ratio.head()

age_ratio.loc[:,'Ch1'] = age_ratio['Ch1'].str.replace('\'','').astype(float)
age_ratio['Active'] = round(age_ratio['Ch1'].div(age_ratio['Rate'])*100,0)

age_1_active = list(age_ratio.loc[(age_ratio['age class'] == 1)].sort_index(ascending = False)['Active'])
age_2_active = list(age_ratio.loc[(age_ratio['age class'] == 2)].sort_index(ascending = False)['Active'])
age_3_active = list(age_ratio.loc[(age_ratio['age class'] == 3)].sort_index(ascending = False)['Active'])

In [ ]:
assumption_2 = age_nationality_to_plot.copy()
assumption_2.columns = ['ch1','ch2','ch3','ch4','ch5','ch6']

assumption_2.loc[:,['ch1','ch4']] = assumption_2[['ch1','ch4']].div(age_1_active, axis =0)*2
assumption_2.loc[:,['ch2','ch5']] = assumption_2[['ch2','ch5']].div(age_2_active, axis =0)*2
assumption_2.loc[:,['ch3','ch6']] = assumption_2[['ch3','ch6']].div(age_3_active, axis =0)*2

assumption_2_to_plot = assumption_2.copy()
assumption_2_to_plot.columns = header
assumption_2_to_plot.columns.names = ['Origin', 'Age']

assumption_2_to_plot.head()

In [ ]:
assumption_2_to_plot.iloc[0:13].plot(kind='bar',figsize=[20,8],fontsize=17, grid = True, \
                                        title='Unemployment in all 26 Swiss Cantons, for nationality class and age class',\
                                        colormap = 'bwr', rot = 0)
assumption_2_to_plot.iloc[14::].plot(kind='bar',figsize=[20,8],fontsize=17, grid = True,\
                                        title='Unemployment in all 26 Swiss Cantons, for nationality class and age class',\
                                        colormap = 'bwr', rot = 0)
plt.show()

From this plot, we can identify global unemployment tendency. Here we have considered the active population is equal for a given age class in both foreign and suisse cathegories.  
We notice unemployment rates are higher for age 15-24 for swiss residents and for age 25-49 for foreign residents. 
Of course this model is very limited because the range swiss residents might have the larger working population than foreign residents. 

# 4 Bonus

In [ ]:
French = ['JU','GE','VD','FR','VS','NE','BE']
Deutch = ['ZH','LU','UR','SZ','OW','NW','GL','ZG','SO','BS','BL','SH','AR','AI','SG','AG','TG','GR']
Italian = ['TI']

rostigraben = pd.DataFrame()
rostigraben['Rates'] = foreign_suisse[['Suisse','Foreign']].mean(axis = 1)
rostigraben['Swiss Rates'] = foreign_suisse[['Suisse']]
rostigraben['Foreign Rates'] = foreign_suisse[['Foreign']]

rostigraben['Canton']= foreign_suisse['Abrev']
rostigraben.set_index('Canton', drop = False, inplace = True)

rostigraben.loc[French,['Rates','Swiss Rates','Foreign Rates']] = [rostigraben.loc[French,['Rates','Swiss Rates','Foreign Rates']].mean(axis = 0)]*len(French)
rostigraben.loc[Deutch,['Rates','Swiss Rates','Foreign Rates']] = [rostigraben.loc[Deutch,['Rates','Swiss Rates','Foreign Rates']].mean(axis = 0)]*len(Deutch)
rostigraben.loc[Italian,['Rates','Swiss Rates','Foreign Rates']] = [rostigraben.loc[Italian,['Rates','Swiss Rates','Foreign Rates']].mean(axis = 0)]*len(Italian)


In [ ]:
m_rostigraben_total = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)

m_rostigraben_total.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons', data=rostigraben.loc[canton_order_map,:],
             columns=['Canton', 'Rates'],
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9,
             legend_name='Unemployement ratios')

m_rostigraben_total.save('rostigraben_total.html')

In [ ]:
m_rostigraben_suisse = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)

m_rostigraben_suisse.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons', data=rostigraben.loc[canton_order_map,:],
             columns=['Canton', 'Swiss Rates'],
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9,
             legend_name='Unemployement ratios')

m_rostigraben_suisse.save('rostigraben_suisse.html')

In [ ]:
m_rostigraben_foreign = folium.Map([47,8.5], tiles='cartodbpositron', min_zoom = 8, max_zoom  = 8)

m_rostigraben_foreign.choropleth(geo_data=canton_topojson, topojson = 'objects.cantons', data=rostigraben.loc[canton_order_map,:],
             columns=['Canton', 'Foreign Rates'],
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.9,
             legend_name='Unemployement ratios')

m_rostigraben_foreign.save('rostigraben_foreign.html')

[Rostigraben Total Unemployement rates](rostigraben_total.html)     
[Rostigraben Swiss Unemployement rates](rostigraben_suisse.html)     
[Rostigraben Foreign Unemployement rates](rostigraben_Foreign.html)     

We notice Italian and German speaking regions have very close unemployment rates in both Swiss and Foreign categories